In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig


c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from numba import jit,cuda
@jit
def read_data():
    test_csv = pd.read_csv('datasets/cleaned_test_data.csv')
    train_csv = pd.read_csv('datasets/clean_train.csv')
    test_csv = test_csv.drop(test_csv.index[0])
    return train_csv,test_csv
df_train,df_test = read_data()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_13184\2875405556.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def read_data():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_13184\2875405556.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "read_data" failed type inference due to: Unknown attribute 'read_csv' of type Module(<module 'pandas' from 'c:\\Users\\Jerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pandas\\__init__.py'>)

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_13184\2875405556.py", line 4:
<source missing, REPL/exec in use?>

During: typing of get attribute at C:\Users\Jerry\Ap

In [3]:
df_balanced = df_train[df_train['class']==1].sample(100)
for index in range(2,11):
    df_balanced = pd.concat([df_balanced,df_train[df_train['class']==index].sample(100)])
x_train = df_balanced['text']
labels = df_balanced['class']

In [17]:
max_len=256
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout= Dropout(0.5)(dense)
    pred = Dense(11, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [3]:
print('Loading the saved pickle files..')
pickle_inp_path='./data/dbert_inp.pkl'
pickle_mask_path='./data/dbert_mask.pkl'
pickle_label_path='./data/dbert_label.pkl'
input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))
print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

Loading the saved pickle files..
Input shape (20000, 32) Attention mask shape (20000, 32) Input label shape (20000,)


In [18]:
model_save_path='./dbert_model.h5'
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
trained_model = create_model()
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 tf_distil_bert_model_4 (TFDist  TFBaseModelOutput(l  66362880   ['input_6[0][0]',                
 ilBertModel)                   ast_hidden_state=(N               'input_7[0][0]']                
                                one, 256, 768),                                                   
                                 hidden_states=None                                         

In [21]:
from IPython.display import clear_output
while(True):
    clear_output(wait=True)
    string = input()
    if(string == 'exit'):break
    input_list = []
    mask_list=[]
    dbert_inps=dbert_tokenizer.encode_plus(string,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_list.append(dbert_inps['input_ids'])
    mask_list.append(dbert_inps['attention_mask'])
    input_id = np.array(input_list)
    mask = np.array(mask_list)
    preds = trained_model.predict([input_id,mask],batch_size=16)
    print(f"input_id: {input_id}")
    print(f"attention_mask: {mask}")
    print(f"prediction: {preds}")
    pred_labels = preds.argmax(axis=1)
    testing = {1: 'Society & Culture', 2: 'Science & Mathematics', 3: 'Health', 4: 'Education & Reference',
                5: 'Computers & Internet', 6: 'Sports', 7: 'Business & Finance', 8: 'Entertainment & Music',
                9: 'Family & Relationships', 10: 'Politics & Government'}
    print(testing[pred_labels[0]])

1/1 [==============================] - 0s 238ms/step
input_id: [[ 101 2065 2396 2003 2000 2053 9496 4095 1996 6147 1997 2256 3226  102
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
 

In [12]:
type(dbert_tokenizer)

transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer

In [5]:
preds = trained_model.predict([val_inp,val_mask],batch_size=16)
pred_labels = preds.argmax(axis=1)
print('Classification Report')
print(classification_report(val_label,pred_labels))
print('Training and saving built model.....')

250/250 [==============================] - 106s 419ms/step
Classification Report
              precision    recall  f1-score   support

           1       0.84      0.83      0.83       408
           2       0.95      0.87      0.91       416
           3       0.94      0.92      0.93       409
           4       0.83      0.85      0.84       385
           5       0.94      0.96      0.95       394
           6       0.97      0.95      0.96       403
           7       0.85      0.71      0.78       397
           8       0.82      0.90      0.86       382
           9       0.82      0.94      0.88       399
          10       0.91      0.95      0.93       407

    accuracy                           0.89      4000
   macro avg       0.89      0.89      0.89      4000
weighted avg       0.89      0.89      0.89      4000

Training and saving built model.....


In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(trained_model)
tflite_model = converter.convert()
open("distilbert_slim_model.tflite","wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Jerry\AppData\Local\Temp\tmpoeskzeh2\assets


INFO:tensorflow:Assets written to: C:\Users\Jerry\AppData\Local\Temp\tmpoeskzeh2\assets


266415768

In [15]:
val_inp[0]

array([  101,  2175,  6998,  3789,  3924,  6830,  2156,  6433,  3134,
        2051,  1029,  3437, 14477,  3619, 13777,  2094,  3980,  3789,
        3437,  2190,  2156,  4148,  3134,  2051,  1012,  1012,  1032,
        1050,  1032,  1050,  2903,   102])

In [36]:
# categories = {'Society & Culture' :1,'Science & Mathematics':2,'Health':3, 'Education & Reference':4,
#             'Computers & Internet' :5,'Sports' :6,'Business & Finance' :7,'Entertainment & Music' : 8,
#             'Family & Relationships':9, 'Politics & Government':10}
# df_train['class']= df_train['class'].map(categories)
df_train

,text,class
0,nt optical mouse work glass table even surfac...,5
1,best offroad motorcycle trail longdistance tr...,6
2,trans fat reduce heard tras fat bad body f...,3
3,many planes fede heard largest airline world ...,7
4,san francisco bay area make sense rent buy ...,7
...,...,...
768320,believe hopelessness believe religion lack ...,1
768321,get horse s skeletal muscular systems web prin...,4
768322,quest promote racial equality government media...,10
768323,ways sell video games like want sell video ga...,7


In [6]:
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [5]:
x_train = df_balanced['text']
y_train = df_balanced['class']

In [6]:
max_len=256
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout= Dropout(0.5)(dense)
    pred = Dense(11, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model

In [7]:
model = create_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 256, 768),                                                   
                                 hidden_states=None                                           

In [8]:
input_ids=[]
attention_masks=[]

for sent in x_train:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [77]:

print('Preparing the pickle file.....')

pickle_inp_path='./data/dbert_inp.pkl'
pickle_mask_path='./data/dbert_mask.pkl'
pickle_label_path='./data/dbert_label.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

Preparing the pickle file.....
Pickle files saved as  ./data/dbert_inp.pkl ./data/dbert_mask.pkl ./data/dbert_label.pkl


In [3]:
pickle_inp_path='./data/dbert_inp.pkl'
pickle_mask_path='./data/dbert_mask.pkl'
pickle_label_path='./data/dbert_label.pkl'

In [4]:
print('Loading the saved pickle files..')
pickle_inp_path='./data/dbert_inp.pkl'
pickle_mask_path='./data/dbert_mask.pkl'
pickle_label_path='./data/dbert_label.pkl'
input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (20000, 32) Attention mask shape (20000, 32) Input label shape (20000,)


In [9]:
log_dir='dbert_model_new'
model_save_path='./dbert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

In [10]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)


In [11]:
history=model
@jit
def modelFitting():
    history.fit([train_inp,train_mask],train_label,batch_size=16,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)
modelFitting()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_13184\1279885436.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def modelFitting():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_13184\1279885436.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "modelFitting" failed type inference due to: Untyped global name 'history': Cannot determine Numba type of <class 'keras.engine.functional.Functional'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_13184\1279885436.py", line 4:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py

Epoch 1/5


C:\Users\Jerry\AppData\Roaming\Python\Python310\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


50/50 [==============================] - 444s 9s/step - loss: 8.5395 - accuracy: 0.1775 - val_loss: 7.9912 - val_accuracy: 0.4200
Epoch 2/5
50/50 [==============================] - 462s 9s/step - loss: 7.4774 - accuracy: 0.5813 - val_loss: 7.3016 - val_accuracy: 0.6500
Epoch 3/5
50/50 [==============================] - 488s 10s/step - loss: 6.8071 - accuracy: 0.7650 - val_loss: 7.0464 - val_accuracy: 0.6650
Epoch 4/5
50/50 [==============================] - 474s 9s/step - loss: 6.4278 - accuracy: 0.8575 - val_loss: 7.1091 - val_accuracy: 0.6750
Epoch 5/5
50/50 [==============================] - 482s 10s/step - loss: 6.1467 - accuracy: 0.9038 - val_loss: 7.0391 - val_accuracy: 0.6550


In [12]:
x_test = df_test['text']
y_test = df_test['class']

In [26]:
import re
df_clean=df_train
@jit
def clean_df():
    df_clean['text'] = df_clean['text'].map(clean_text)
clean_df()
    


C:\Users\Jerry\AppData\Local\Temp\ipykernel_24472\2584200295.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def clean_df():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_24472\2584200295.py:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "clean_df" failed type inference due to: Untyped global name 'df_clean': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_24472\2584200295.py", line 5:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWar

In [34]:
df_clean.to_csv('datasets/clean_train.csv', index=False)

In [23]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    return text

In [79]:
x_train.values

array(['people still offended michael richards remarks  ok   maybe s racist definetely nt said things said damn   get  even racist   hypothetical     going impact people offended  yeah',
       'question christians  jesus reworked pagan myth early 1st century jews   secular evidence lived crucified ',
       'ever sing loud  ever dance street  ndo ever say cheery  hello  stranger  nwhat result  think best thing know try  s nothing best  first hand eperience  know    smile  ',
       ...,
       'question people believe last question please answer  read  tell feel  honest opinions helpthis still happening believe notnsoldiers stories n   2 700 could fulfill full 36month tour duty   ve got pay back advance nt finish   wwwhendersoncountyncdemocratsorg soldiers_storieshtml  53k  cached  site simple accounting screwup  nnif gone military first instead news media would fied right away ',
       'prop 83 mess california  voters epect rehabilatated se offenders kicked immediately voting   appl

In [93]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(x_train.values)
X = tokenizer.texts_to_sequences(x_train.values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
Y = pd.get_dummies(y_train).values


In [94]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)

In [95]:
Y_train.shape

(180000, 10)

In [64]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,SpatialDropout1D, LSTM
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [65]:
from tensorflow.keras.callbacks import EarlyStopping
epochs = 5
batch_size = 64
history = model
@jit
def modelFitting():
    history.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
modelFitting()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_24472\2224690945.py:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def modelFitting():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_24472\2224690945.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "modelFitting" failed type inference due to: Untyped global name 'history': Cannot determine Numba type of <class 'keras.engine.sequential.Sequential'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_24472\2224690945.py", line 7:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py

Epoch 1/5
254/254 [==============================] - 293s 1s/step - loss: 1.9050 - accuracy: 0.3202 - val_loss: 1.5366 - val_accuracy: 0.4800
Epoch 2/5
254/254 [==============================] - 306s 1s/step - loss: 1.1295 - accuracy: 0.6278 - val_loss: 1.3392 - val_accuracy: 0.5683
Epoch 3/5
254/254 [==============================] - 301s 1s/step - loss: 0.7602 - accuracy: 0.7665 - val_loss: 1.2250 - val_accuracy: 0.6378
Epoch 4/5
254/254 [==============================] - 310s 1s/step - loss: 0.4725 - accuracy: 0.8618 - val_loss: 1.3652 - val_accuracy: 0.6183
Epoch 5/5
254/254 [==============================] - 315s 1s/step - loss: 0.3210 - accuracy: 0.9101 - val_loss: 1.3908 - val_accuracy: 0.6272


In [66]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

63/63 [==============================] - 5s 73ms/step - loss: 1.4505 - accuracy: 0.6135
Test set
  Loss: 1.451
  Accuracy: 0.613


Test trained model

In [17]:
val_inp

array([[  101,  9530, 21163, ...,  7419,  5350,   102],
       [  101,  5788, 29500, ...,  3132, 27668,   102],
       [  101,  6898,  2183, ..., 17612,  2015,   102],
       ...,
       [  101,  1029,  2003, ...,  5604,  4255,   102],
       [  101,  6912,  3160, ...,  9157,  6187,   102],
       [  101,  3728,  2318, ...,  2204, 18012,   102]])

In [96]:
from tensorflow.keras.layers import SimpleRNN
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, 24, input_length=X.shape[1]))
model.add(SimpleRNN(24, return_sequences=False))
model.add(Dense(10, activation='sigmoid'))

# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [98]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# fit the model
history = model
@jit
def modelRNN():
    history.fit(x=X_train,
         y=Y_train,
         epochs=10,
         validation_data=(X_test, Y_test), verbose=1,
         callbacks=[early_stop]
         )
modelRNN()

Epoch 1/10


C:\Users\Jerry\AppData\Local\Temp\ipykernel_24472\4164863293.py:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def modelRNN():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_24472\4164863293.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "modelRNN" failed type inference due to: Untyped global name 'history': Cannot determine Numba type of <class 'keras.engine.sequential.Sequential'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_24472\4164863293.py", line 7:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: Nu

5625/5625 [==============================] - 229s 41ms/step - loss: 0.2065 - accuracy: 0.5894 - val_loss: 0.1972 - val_accuracy: 0.6189
Epoch 2/10
5625/5625 [==============================] - 229s 41ms/step - loss: 0.1834 - accuracy: 0.6522 - val_loss: 0.1875 - val_accuracy: 0.6534
Epoch 3/10
5625/5625 [==============================] - 226s 40ms/step - loss: 0.1739 - accuracy: 0.6751 - val_loss: 0.1814 - val_accuracy: 0.6565
Epoch 4/10
5625/5625 [==============================] - 221s 39ms/step - loss: 0.1672 - accuracy: 0.6897 - val_loss: 0.1799 - val_accuracy: 0.6647
Epoch 5/10
5625/5625 [==============================] - 220s 39ms/step - loss: 0.1631 - accuracy: 0.6998 - val_loss: 0.1924 - val_accuracy: 0.6413
Epoch 6/10
5625/5625 [==============================] - 220s 39ms/step - loss: 0.1600 - accuracy: 0.7061 - val_loss: 0.1761 - val_accuracy: 0.6756
Epoch 7/10
5625/5625 [==============================] - 220s 39ms/step - loss: 0.1574 - accuracy: 0.7104 - val_loss: 0.1787 - val

In [31]:
preds = trained_model.predict([val_inp,val_mask],batch_size=32)
pred_labels = preds.argmax(axis=1)
print('Classification Report')
print(classification_report(val_label,pred_labels))

print('Training and saving built model.....')   

12/63 [====>.........................] - ETA: 1:26

18/63 [=======>......................] - ETA: 1:19

Classification Report
              precision    recall  f1-score   support

           1       0.90      0.90      0.90       393
           2       0.92      0.96      0.94       420
           3       0.95      0.94      0.95       400
           4       0.89      0.88      0.89       427
           5       0.97      0.96      0.96       406
           6       0.96      0.97      0.96       378
           7       0.88      0.86      0.87       390
           8       0.95      0.91      0.93       388
           9       0.93      0.95      0.94       407
          10       0.93      0.95      0.94       391

    accuracy                           0.93      4000
   macro avg       0.93      0.93      0.93      4000
weighted avg       0.93      0.93      0.93      4000

Training and saving built model.....


In [91]:
val_mask

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]])